In [1]:
import pandas as pd
import numpy as np
from data_preprocessing import *

set_names = ['INDU-HAND-RAND',
             'SAT_Competition_RACE_HAND',
             'SAT_Competition_RACE_RAND',
             'SAT_Competition_RACE_INDU',
             'IBM-SWV',
             'IBM-ALL',
             'SWV',
             'SAT_Competition_RACE_RAND_SAT']

# ovo je skup za koji smo diskutovali visoku vrednost varijanse objasnjene jednom PCA komponentom
set_name = set_names[0] 

X = pd.read_csv('data/' + 'SAT_Competition_RACE_RAND_SAT' + '-feat.csv')
Y = pd.read_csv('data/' + 'SAT_Competition_RACE_RAND_SAT' + '-' + 'tnm' + '.csv',
                  names=['INSTANCE_ID', 'SOLVER_TIME'])
print(X['INSTANCE_ID'].equals(Y['INSTANCE_ID']))

X = pd.read_csv('data/gcrf/' + 'INDU-IBM-SWV' + '-feat.csv')
Y = pd.read_csv('data/gcrf/' + 'INDU-IBM-SWV' + '-' + 'spear' + '.csv',
                  names=['INSTANCE_ID', 'SOLVER_TIME'])
print(X['INSTANCE_ID'].equals(Y['INSTANCE_ID']))
# X = X.drop('INSTANCE_ID', axis=1).get_values()

# print(X.shape)
# X[X==512]

True
True


In [41]:
np.linalg.cond(X.T.dot(X))

2.0446487801116712e+21

In [42]:
# uslovljenosti svih matrica
for set_name in set_names:
    X = pd.read_csv('data/' + set_name + '-feat.csv')
    X = X.drop('INSTANCE_ID', axis=1).get_values()
    print(set_name, ':', np.linalg.cond(X.T.dot(X)))

INDU-HAND-RAND : 2.04464878011e+21
SAT_Competition_RACE_HAND : 3.80260569124e+20
SAT_Competition_RACE_RAND : 2.08134638123e+28
SAT_Competition_RACE_INDU : 8.97181055379e+30
IBM-SWV : 7.0892524149e+31
IBM-ALL : 1.67809590413e+33
SWV : 5.51894584938e+33
SAT_Competition_RACE_RAND_SAT : 4.69748494948e+28


In [45]:
# broj instanci pre i nakon izbacivanja onih sa ostecenim vrednostima atributa
for set_name in set_names:
    X = pd.read_csv('data/' + set_name + '-feat.csv')
    X = X.drop('INSTANCE_ID', axis=1).get_values()
    print(set_name, ':', X.shape[0], X[~(X == -512).any(axis=1)].shape[0])

INDU-HAND-RAND : 7012 2802
SAT_Competition_RACE_HAND : 1955 765
SAT_Competition_RACE_RAND : 3381 1867
SAT_Competition_RACE_INDU : 1676 170
IBM-SWV : 1369 22
IBM-ALL : 765 8
SWV : 604 14
SAT_Competition_RACE_RAND_SAT : 2076 1140


In [121]:
def S(Y, delta=1, svd=False, scale=False):
  num_solvers = Y.shape[1]
  S = np.zeros((num_solvers, num_solvers))
  if svd == True:
    U, _sigma, _V = np.linalg.svd(Y.T)
  min = max = 0
  for k in range(0, num_solvers):
    for l in range(0, num_solvers):
      if k == l: continue
      if svd == True:
        print(U[k, :])
        print(U[l, :])
        print(np.linalg.norm(U[k, :] - U[l, :]) ** 2)
        print("\n")
        S[k, l] = np.exp(- delta * np.linalg.norm(U[k, :] - U[l, :]) ** 2)
      else:
        print(np.linalg.norm(Y[:, k] - Y[:, l]) ** 2)
        print("\n")
        S[k, l] = np.exp(- delta * np.linalg.norm(Y[:, k] - Y[:, l]) ** 2)
      S[l, k] = S[k, l]
      if S[k, l] > max: max = S[k, l]
  if scale:
    print(min)
    print(max)
    # scaling to [0, 1]
    if max > 0: S = (S - min) / (max - min)
  return S

In [122]:
dataset = 'RAND_SAT'
# dataset = 'INDU-IBM-SWV'
Y = pd.read_csv('data/gcrf/' + dataset + '-results.csv').drop('INSTANCE_ID', axis=1).get_values()

In [131]:
S(Y, delta=9e-11, svd=False, scale=True)

15554485794.6


10136255158.7


15554485794.6


5477486059.65


10136255158.7


5477486059.65


0
0.610807305376


array([[ 0.        ,  0.40376188,  0.657514  ],
       [ 0.40376188,  0.        ,  1.        ],
       [ 0.657514  ,  1.        ,  0.        ]])